In [32]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from More_futuristic_LSTM import*

In [33]:
df_prices = pd.read_excel('data/train.xlsx', usecols="B:Z")

C:\Users\anton\AppData\Local\Temp\ipykernel_9036\47752094.py:1: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df_prices = pd.read_excel('data/train.xlsx', usecols="B:Z")


In [34]:
df_days = pd.read_excel('data/train.xlsx', usecols="A")
df_days["PRICES"] = df_days["PRICES"].dt.day_of_week
df_days["a"], df_days["b"], df_days["c"], df_days["v"], df_days["w"], df_days["d"], df_days["e"], df_days["f"], df_days["g"], df_days["h"], df_days["i"], df_days["j"], df_days["k"], df_days["l"], df_days["m"], df_days["n"], df_days["o"], df_days["p"], df_days["q"], df_days["r"], df_days["s"], df_days["t"], df_days["u"] = [df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"], df_days["PRICES"]]
df_days = df_days.to_numpy().flatten()

In [35]:
df_week = pd.read_excel('data/train.xlsx', usecols="A")
df_week["PRICES"] = df_week["PRICES"].dt.isocalendar().week - 1
df_week["a"], df_week["b"], df_week["c"], df_week["v"], df_week["w"], df_week["d"], df_week["e"], df_week["f"], df_week["g"], df_week["h"], df_week["i"], df_week["j"], df_week["k"], df_week["l"], df_week["m"], df_week["n"], df_week["o"], df_week["p"], df_week["q"], df_week["r"], df_week["s"], df_week["t"], df_week["u"] = [df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"], df_week["PRICES"]]
df_week = df_week.to_numpy().flatten()

In [36]:
df_month = pd.read_excel('data/train.xlsx', usecols="A")
df_month["PRICES"] = df_month["PRICES"].dt.month - 1
df_month["a"], df_month["b"], df_month["c"], df_month["v"], df_month["w"], df_month["d"], df_month["e"], df_month["f"], df_month["g"], df_month["h"], df_month["i"], df_month["j"], df_month["k"], df_month["l"], df_month["m"], df_month["n"], df_month["o"], df_month["p"], df_month["q"], df_month["r"], df_month["s"], df_month["t"], df_month["u"] = [df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"], df_month["PRICES"]]
df_month = df_month.to_numpy().flatten()

In [37]:
df_hour = np.arange(24)
df_hour = np.tile(df_hour, df_prices.shape[0])


In [38]:
df_prices = df_prices.to_numpy().flatten()
len(df_prices)

26304

In [39]:
observations = pd.DataFrame({
    "prices": df_prices, 
    "hour": df_hour,
    "day": df_days,
    "week": df_week,
    "month" : df_month
})

In [40]:
observations['weekend'] = observations['day'].apply(lambda x: 1 if x > 4 else 0)
observations.to_csv('data/train_processed.csv')

In [41]:
with open(f'./LSTMmodel-66.pickle','rb') as f:
    model = pickle.load(f)

train_data = pd.read_csv('data/train_processed.csv')

X, Y = prepare_data(train_data, num_time_steps=27, num_predict_steps=3)
Y_pred = model.predict(X)
action_list = assign_action_points(Y_pred, X)


Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-01-31 01:37:58         1780
metadata.json                                  2023-01-31 01:37:58           64
variables.h5                                   2023-01-31 01:37:58      1117584
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\lstm
......vars
...layers\lstm\cell
......vars
.........0
.........1
.........2
...metrics\mean
......vars
.........0
.........1
...optimizer
......vars
.........0
.........1
.........10
.........2
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
822/822 [==============================] - 4s 5ms/step


c:\Users\anton\py_versions\py3104\lib\site-packages\scipy\optimize\_minpack_py.py:833: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


In [42]:
observations['lstm']=action_list

In [43]:
observations.to_csv('data/train_processed_wth_lstm.csv')
observations['lstm'].value_counts()

 0    15024
 1     5964
-1     5316
Name: lstm, dtype: int64